<a href="https://colab.research.google.com/github/hucarlos08/Mathematics/blob/main/Labs/ExampleGGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentinel-2 Satellite Image Analysis with Google Earth Engine

This notebook demonstrates how to access, visualize and analyze Sentinel-2
satellite imagery using Google Earth Engine (GEE) and Python. Unlike traditional
approaches that require downloading satellite data locally, GEE allows us to
process satellite imagery in the cloud.

The notebook covers:
1. Basic setup and authentication
2. Loading and visualizing Sentinel-2 imagery
3. Understanding and visualizing individual bands
4. Creating different band compositions
5. Calculating spectral indices

In [ ]:
# Required imports
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Initialize Earth Engine with your project
project_id = 'ee-hcarlos-sentinel2'  # Replace with your GEE project ID

try:
    ee.Initialize(project=project_id)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=project_id)


## Sentinel-2 Band Information

Sentinel-2 provides 13 spectral bands:
- 4 bands at 10m: Blue (B2), Green (B3), Red (B4), NIR (B8)
- 6 bands at 20m: Red Edge (B5, B6, B7, B8A), SWIR (B11, B12)
- 3 bands at 60m: Coastal aerosol (B1), Water vapor (B9), SWIR-Cirrus (B10)


In [ ]:
import pandas as pd


SENTINEL2_BANDS_INFO = {
    'B1': {'description': 'Coastal aerosol', 'resolution': 60, 'wavelength': '443nm'},
    'B2': {'description': 'Blue', 'resolution': 10, 'wavelength': '490nm'},
    'B3': {'description': 'Green', 'resolution': 10, 'wavelength': '560nm'},
    'B4': {'description': 'Red', 'resolution': 10, 'wavelength': '665nm'},
    'B5': {'description': 'Red Edge 1', 'resolution': 20, 'wavelength': '705nm'},
    'B6': {'description': 'Red Edge 2', 'resolution': 20, 'wavelength': '740nm'},
    'B7': {'description': 'Red Edge 3', 'resolution': 20, 'wavelength': '783nm'},
    'B8': {'description': 'NIR', 'resolution': 10, 'wavelength': '842nm'},
    'B8A': {'description': 'Red Edge 4', 'resolution': 20, 'wavelength': '865nm'},
    'B9': {'description': 'Water vapor', 'resolution': 60, 'wavelength': '940nm'},
    'B11': {'description': 'SWIR 1', 'resolution': 20, 'wavelength': '1610nm'},
    'B12': {'description': 'SWIR 2', 'resolution': 20, 'wavelength': '2190nm'}
}

sentinel2_df = pd.DataFrame.from_dict(SENTINEL2_BANDS_INFO, orient='index')
sentinel2_df

 ## Helper Functions

In [ ]:
def get_sentinel2_collection(start_date, end_date, roi):
    """
    Retrieves Sentinel-2 imagery for a specific time period and region.

    Parameters:
    -----------
    start_date: str
        Start date in 'YYYY-MM-DD' format
    end_date: str
        End date in 'YYYY-MM-DD' format
    roi: ee.Geometry
        Region of interest

    Returns:
    --------
    ee.ImageCollection
        Filtered collection of Sentinel-2 images
    """
    return (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(start_date, end_date)
        .filterBounds(roi)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

def visualize_band(image, band, layer_name=None):
    """
    Visualizes a single Sentinel-2 band on the map.

    Parameters:
    -----------
    image: ee.Image
        Sentinel-2 image
    band: str
        Band name (e.g., 'B4')
    layer_name: str, optional
        Custom name for the layer
    """
    if layer_name is None:
        layer_name = f"Band {band} - {SENTINEL2_BANDS_INFO[band]['description']}"

    vis_params = {
        'min': 0,
        'max': 3000,
        'bands': [band],
        'palette': ['black', 'white']
    }

    Map.addLayer(image, vis_params, layer_name)

## Create Base Map


In [ ]:
Map = geemap.Map()

# Define region of interest (example: Mexico City)
roi = ee.Geometry.Point([-99.1332, 19.4326]).buffer(10000)

 ## Load and Display Sentinel-2 Imagery

In [ ]:
# Define time period
start_date = '2024-01-01'
end_date = '2024-02-01'

# Get image collection and select first image
collection = get_sentinel2_collection(start_date, end_date, roi)
image = collection.first()

# Center map on our region of interest
Map.centerObject(roi, 11)

## Visualization Examples

In [ ]:
# True Color Composite (RGB)
vis_params_rgb = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.addLayer(image, vis_params_rgb, 'RGB Natural Color')

# False Color Infrared (NIR, Red, Green)
vis_params_false_color = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3']
}
Map.addLayer(image, vis_params_false_color, 'False Color (NIR)')

# SWIR Composite
vis_params_swir = {
    'min': 0,
    'max': 3000,
    'bands': ['B12', 'B8A', 'B4']
}
Map.addLayer(image, vis_params_swir, 'SWIR Composite')

# Visualize all individual bands
for band in SENTINEL2_BANDS_INFO.keys():
    visualize_band(image, band)

# Display the map
Map

## Spectral Indices

Spectral indices are combinations of surface reflectance at two or more
wavelengths that indicate relative abundance of features of interest.

Common indices include:

1. NDVI (Normalized Difference Vegetation Index):
   $NDVI = \frac{NIR - Red}{NIR + Red}$

2. SAVI (Soil Adjusted Vegetation Index):
   $SAVI = \frac{NIR - Red}{NIR + Red + L} \times (1 + L)$
   where L is a soil brightness correction factor

3. NDMI (Normalized Difference Moisture Index):
   $NDMI = \frac{NIR - SWIR1}{NIR + SWIR1}$

4. MNDWI (Modified Normalized Difference Water Index):
   $MNDWI = \frac{Green - SWIR}{Green + SWIR}$

5. VARI (Visible Atmospherically Resistant Index):
   $VARI = \frac{Green - Red}{Green + Red - Blue}$

In [ ]:
def calculate_indices(image):
    """
    Calculate various spectral indices from a Sentinel-2 image

    Parameters:
    -----------
    image: ee.Image
        Sentinel-2 image

    Returns:
    --------
    ee.Image
        Image with added index bands
    """
    # NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # SAVI (with L=0.5 for intermediate vegetation cover)
    L = 0.5
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': L
        }
    ).rename('SAVI')

    # NDMI
    ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')

    # MNDWI
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')

    # VARI
    vari = image.expression(
        '(GREEN - RED) / (GREEN + RED - BLUE)',
        {
            'GREEN': image.select('B3'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('VARI')

    return image.addBands([ndvi, savi, ndmi, mndwi, vari])

# Calculate all indices
image_with_indices = calculate_indices(image)

# Visualization parameters for indices
INDEX_VIS_PARAMS = {
    'NDVI': {
        'min': -1,
        'max': 1,
        'palette': ['blue', 'white', 'green']
    },
    'SAVI': {
        'min': -1,
        'max': 1,
        'palette': ['brown', 'yellow', 'green']
    },
    'NDMI': {
        'min': -1,
        'max': 1,
        'palette': ['red', 'white', 'blue']
    },
    'MNDWI': {
        'min': -1,
        'max': 1,
        'palette': ['brown', 'white', 'blue']
    },
    'VARI': {
        'min': -1,
        'max': 1,
        'palette': ['red', 'white', 'green']
    }
}

# Create a new map
Map2 = geemap.Map()
Map2.centerObject(roi, 11)

# Add each index to the map
for index_name, vis_params in INDEX_VIS_PARAMS.items():
    Map2.addLayer(
        image_with_indices.select(index_name),
        vis_params,
        f'{index_name} Index'
    )

Map2

 ## Additional Band Compositions

 Special band compositions can highlight specific features:

1. Agriculture: NIR, SWIR1, Blue (B8, B11, B2)
2. Atmospheric Penetration: SWIR2, SWIR1, NIR (B12, B11, B8)
3. Vegetation Analysis: NIR, Red Edge, Red (B8, B5, B4)
4. Healthy Vegetation: NIR, Red Edge 3, Red Edge 1 (B8, B7, B5)

In [ ]:
# Agriculture
vis_params_agriculture = {
    'bands': ['B8', 'B11', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}
Map2.addLayer(image, vis_params_agriculture, 'Agriculture Composite')

# Atmospheric Penetration
vis_params_atmospheric = {
    'bands': ['B12', 'B11', 'B8'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}
Map2.addLayer(image, vis_params_atmospheric, 'Atmospheric Penetration')

# Vegetation Analysis
vis_params_vegetation = {
    'bands': ['B8', 'B5', 'B4'],
    'min': 0,
    'max': 3000
}
Map2.addLayer(image, vis_params_vegetation, 'Vegetation Analysis')

# Healthy Vegetation
vis_params_healthy_veg = {
    'bands': ['B8', 'B7', 'B5'],
    'min': 0,
    'max': 3000
}
Map2.addLayer(image, vis_params_healthy_veg, 'Healthy Vegetation')
